# Otimização de Markowitz

## Índice Sharpe

Mede a rentabilidade em relação à volatilidade da carteira. É normalmente apresentado junto com a volatilidade líquida e a rentabilidade líquida.

Se considerarmos o CDI como ativo livre de risco, a fórmula fica:

<div style="text-align: left">

$$
\frac{\text{rentabilidade do ativo} - \text{CDI}}{\text{volatilidade do ativo}}
$$

</div>


Em alguns casos, podemos desconsiderar o ativo livre de risco. Por exemplo:

Se o retorno foi 20% e a volatilidade foi 10%, o Sharpe do investimento é 2.

---

### Fórmula canônica

<div style="text-align: left">

$$
\text{Sharpe} = \frac{R_p - R_f}{\sigma_p}
$$

</div>

* \(R_p\) — retorno médio do ativo ou portfólio  
* \(R_f\) — taxa livre de risco (ex.: CDI, T‑bill)  
* \(\sigma_p\) — volatilidade (desvio padrão) dos retornos de \(R_p\)

Essa fórmula mede o **retorno excedente por unidade de risco total**.

---

### Quando se usa \(R_f = 0\)

| Situação | Por quê? |
| --- | --- |
| Exemplos didáticos | Para simplificar a explicação e a matemática. |
| Séries já em *excess return* | O retorno livre de risco já foi subtraído. |
| Frequência diária ou intradiária | A taxa diária do CDI é desprezível. |
| Ambientes com juro ≈ 0% | Comuns após 2008 em EUA, Europa ou Japão. |
| Estratégias *self-financing* | O retorno já é líquido de funding (ex.: market-neutral). |

---

### Exemplo

* Retorno anual do ativo: **20%**  
* CDI anual: **12%**  
* Volatilidade anual: **10%**

<div style="text-align: left">

$$
\text{Sharpe} = \frac{0.20 - 0.12}{0.10} = 0.8
$$

</div>

*Se \(R_f = 0\), o mesmo caso daria Sharpe = 2.*

---

## Como funciona a otimização

O objetivo da otimização de Markowitz é **encontrar a carteira com o maior índice de Sharpe**.

Por quê?  
Porque quanto maior o Sharpe, **maior o retorno esperado para cada unidade de risco** (volatilidade) assumido. Isso significa melhor eficiência na relação risco-retorno.
